In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import math

In [ ]:
df = pd.read_csv('/kaggle/input/used-cars-from-mercadolibre/cars.csv')

In [ ]:
# me quedo solo con los usados
df = df[df.used=='Usado']
df.drop(columns='used',inplace=True)

In [ ]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

In [ ]:
# modelos mas comunes
most_common_models = df[['model','id']].groupby('model').count().reset_index().sort_values(by='id',ascending=False)

In [ ]:
# me quedo con los autos de los 100 modelos mas frecuentes, no me interesa entrenar con aquellos que ponen cualquier fruta en modelo
df = df[df.model.isin(most_common_models.head(100).model.values)]

In [ ]:
df.info()

In [ ]:
df_human = df.copy()

In [ ]:
# entreno con columnas engine_displacement, vehicle_year, model (one hot), doors, km, transm (one hot) -> price

In [ ]:
df = df[['price','engine_displacement','vehicle_year','doors','km','model']]

In [ ]:
df = encode_and_bind(df, 'model')

In [ ]:
df.drop(columns='model',inplace=True)

In [ ]:
def split_vals(a,n):
    return a[:n],a[n:]

In [ ]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res =[rmse(m.predict(X_train),y_train), rmse(m.predict(X_valid),y_valid), m.score(X_train,y_train), m.score(X_valid,y_valid)]
    if hasattr(m,'oob_score_'):res.append(m.oob_score_)
    print(res)

In [ ]:
df_raw_train, df_raw_test = train_test_split(df)

In [ ]:
n_valid = 100
n_train = len(df_raw_train)-n_valid
X_train,X_valid = split_vals(df_raw_train.drop('price',axis=1),n_train)
y_train,y_valid = split_vals(df_raw_train['price'],n_train)
X_test = df_raw_test

In [ ]:
m = RandomForestRegressor(n_jobs=1, oob_score=True)

In [ ]:
m.fit(X_train, y_train)

In [ ]:
print_score(m)

In [ ]:
def feat_importance(m,df_train):
    importance = m.feature_importances_
    importance = pd.DataFrame(importance,index=df_train.columns,columns=["Importance"])
    return importance.sort_values(by=['Importance'],ascending=False)
importance = feat_importance(m,X_train)
#importance[:]

In [ ]:
df_human['predicted'] = m.predict(df.drop(columns='price'))

In [ ]:
df_human['diff'] = df_human.apply(lambda row: abs(row['price'] - row['predicted']), axis=1)

In [ ]:
df_human[['brand', 'model','doors','km','price', 'predicted', 'diff']].sort_values(by='diff',ascending=True).head(20)

In [ ]:
df_human[['diff']].describe()

In [ ]:
df2 = df_human